In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)

In [ ]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y%m%d")

noaa_ghcnd_file_path = './data/noaa_ghcn_aws_data/'

file_name = '2021'
noaa_ghcn_aws_data_2021 = pd.read_csv(
    f"{noaa_ghcnd_file_path}{file_name}.csv", 
    header=None, 
    usecols=[0,1,2,3],
    names=['station_id', 'date', 'data_type_id', 'data_value'],#, 'measurement_id', 'quality_id', 'source_id', 'observation_time'],
    nrows=100000, 
    parse_dates=['date'],
    date_parser=custom_date_parser,
    # dtype={'observation_time':str}
    )
#noaa_ghcn_aws_data_2021.observation_time.replace({np.nan: '0000', '2400': '0000'}, inplace=True)
#noaa_ghcn_aws_data_2021.observation_time = pd.to_datetime(noaa_ghcn_aws_data_2021.observation_time, format='%H%M').dt.time

# Extract country code from station_id
noaa_ghcn_aws_data_2021['noaa_country_code'] = noaa_ghcn_aws_data_2021.station_id.str[:2]
# Drop countries that are not in the fao dataset
noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.noaa_country_code.isin(noaa_country_codes.noaa_country_code)]
# Drop unnecessary data types
noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.data_type_id.isin(data_ids_to_keep)]
# Pivot the table.
noaa_weather_data = noaa_ghcn_aws_data_2021.pivot(index=['noaa_country_code', 'station_id', 'date'], columns='data_type_id', values='data_value')
# Fill in the weather type columns with zeroes because they are one-hot encoded/binary
# weather_type_cols = noaa_weather_data.columns[noaa_weather_data.columns.str.contains('WT')]
# noaa_weather_data[weather_type_cols] = noaa_weather_data[weather_type_cols].fillna(0)

noaa_weather_data.reset_index(inplace=True)
noaa_weather_data.head()